with this notebook, I want to:
1. load in the fMRI data from the run
2. extract the mean signal from each ROI in the glasser atlas, along with the ant and post hc in L/R hemi
3. correlate the signals of each pair of ROIs
4. fisher z-transform these signals

## load libraries

In [2]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import numpy.ma as ma
from scipy import stats
import scipy.spatial.distance as sp_distance
from sklearn.svm import NuSVC
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
from sklearn import preprocessing
import seaborn as sns 
from brainiak.isfc import isc
from brainiak.isfc import isfc
from brainiak.fcma.util import compute_correlation
import brainiak.funcalign.srm
#from brainiak import image, io
import pandas as pd
import matplotlib.pyplot as plt
from pandas import *
from nilearn.masking import apply_mask
from nilearn.image import resample_to_img
from nipype.interfaces import afni
from nipy.labs.mask import intersect_masks
from scipy.stats import zscore, norm
from sklearn import linear_model
#from isc_reorder import isc_reorder
#from isc_recall import isc_recall
import pickle
from nilearn import image
import igraph
import bct

from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.formula.api as smf

import multiprocessing as mp
import time

from nilearn.plotting import plot_roi, show


%autosave 5
%matplotlib inline

/Users/alexbarnett/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Autosaving every 5 seconds


## set variables

In [3]:
#data_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/cartoon_recall_fmriprep_2020/fmriprep/'
analysis_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/'
subjlist_full = pd.read_excel(analysis_dir+'subjlist.xlsx')
roi_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/cartoon_recall_fmriprep_2020/freesurfer/HCP-MMP1_subject_atlas/'
roi_list = pd.read_excel(analysis_dir+'roilist.xlsx')
movie_list = ['barmaid','bluestreets']
subj_recall=pd.read_excel(analysis_dir+'subj_recall_sessions_run.xlsx')
event_boundaries = pd.read_excel(analysis_dir+'fmri_recall_event_boundaries_14events_each.xlsx')

In [101]:
glasser_nodes=pd.read_excel(analysis_dir+'within_subj_fc/hc_glasser_nodes.xlsx')

In [4]:
with open(analysis_dir+'subject_design_label_bluestreets.data', 'rb') as filehandle:
        # read the data as binary data stream
        subject_design_label_bluestreets = pickle.load(filehandle)
with open(analysis_dir+'subject_design_label_barmaid.data', 'rb') as filehandle:
        # read the data as binary data stream
        subject_design_label_barmaid = pickle.load(filehandle)

In [5]:
with open(analysis_dir+'barmaid_recall_design.data', 'rb') as filehandle:
    # read the data as binary data stream
    barmaid_design = pickle.load(filehandle)
    
with open(analysis_dir+'bluestreets_recall_design.data', 'rb') as filehandle:
    # read the data as binary data stream
    bluestreets_design = pickle.load(filehandle)

In [6]:
both_movies_design = np.concatenate((barmaid_design,bluestreets_design))

In [7]:
hc_list = ['_L_ant_hippo.nii.gz','_L_post_hippo.nii.gz','_R_ant_hippo.nii.gz','_R_post_hippo.nii.gz']

In [8]:
indsort = np.loadtxt(analysis_dir+'INDSORT.txt')
indsort=indsort.astype('int32')
for i in range(len(indsort)):
    indsort[i] = indsort[i] - 1

In [9]:
## add hippocampal posititions to indsort
indsort_all =np.append(indsort,[358,359,360,361])

In [10]:
indsort_sub = np.loadtxt(analysis_dir+'indsort_subnetworks.txt')
indsort_sub=indsort_sub.astype('int32')
for i in range(len(indsort_sub)):
    indsort_sub[i] = indsort_sub[i] -1

In [11]:
indsort_sub =np.append(indsort_sub,[358,359,360,361])

## load in SRM movie data

In [43]:
# make ROI x TR x subject matrix
num_TR = 550
num_subs = 24
num_glasser_rois = 358
num_hc_rois = 4
num_rois = num_glasser_rois+num_hc_rois
SRM_data =np.empty([num_rois, num_TR, num_subs])

In [49]:
## this is the right preprocessing

def roi_extract(sub):
    SRM_data = np.empty([num_rois,num_TR])
    print(subjlist_full['subj_id'][sub])
    epi_img = nib.load(analysis_dir+'confound_corrected_data/fc_data/'+subjlist_full['subj_id'][sub]+'_'+subjlist_full['ses_num'][sub]+'_task-srmmovie_run-1_space-T1w_desc-preproc_bold_denoised_GSR_filter.nii.gz')
    for roi in range(num_glasser_rois):
        roi_mask = nib.load(roi_dir+subjlist_full['subj_id'][sub]+'/rois/'+roi_list['region'][roi])
        mask_SRM = resample_to_img(roi_mask,epi_img,interpolation='nearest')
        epi_mask_img = apply_mask(epi_img, mask_SRM)
        SRM_data[roi,:] = np.mean(epi_mask_img.T,axis=0)
    return SRM_data
    

In [50]:
## run the parallelized function
start_time = time.time()

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

SRM_data_all=[]
# Step 2: `pool.apply` the `perm_reinstatement()`
SRM_data_all= pool.map(roi_extract, range(num_subs))

# Step 3: Don't forget to close
pool.close() 

print("--- %s seconds ---" % (time.time() - start_time))

sub-101
--- 8.297278881072998 seconds ---


In [25]:
## extract hippocampal ROI data
def roi_extract(sub):
    SRM_data = np.empty([num_hc_rois,num_TR])
    epi_img = nib.load(analysis_dir+'confound_corrected_data/'+subjlist_full['subj_id'][sub]+'_'+subjlist_full['ses_num'][sub]+'_task-srmmovie_run-1_space-T1w_desc-preproc_bold_denoised_GSR_filter.nii.gz')
    for roi in range(num_hc_rois):
        roi_mask = nib.load('/Volumes/dml/ajbarnett/cartoon_recall_fmriprep_2020/hc_masks/'+subjlist_full['subj_id'][sub]+'/'+subjlist_full['subj_id'][sub]+hc_list[roi])
        mask_SRM = resample_to_img(roi_mask,epi_img,interpolation='nearest')
        epi_mask_img = apply_mask(epi_img, mask_SRM)
        SRM_data[roi,:] = np.mean(epi_mask_img.T,axis=0)
    return SRM_data
    

In [26]:
## run the parallelized function
start_time = time.time()

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

SRM_data_hc=[]
# Step 2: `pool.apply` the `perm_reinstatement()`
SRM_data_hc= pool.map(roi_extract, range(num_subs))

# Step 3: Don't forget to close
pool.close() 

print("--- %s seconds ---" % (time.time() - start_time))

--- 37.87176203727722 seconds ---


In [27]:
##combine HC and glasser ROIs

In [28]:
SRM_fc_data = SRM_data_all
for i in range(num_subs): 
    SRM_fc_data[i][358:362][:] = SRM_data_hc[i]

In [29]:
##calculate fc
subj_SRM_fc =[]
for i in range(num_subs):
    temp = SRM_fc_data[i]
    subj_SRM_fc.append(np.corrcoef(temp))

In [78]:
np.save(analysis_dir+'SRM_fc_data_full.npy',SRM_fc_data)

In [79]:
np.save(analysis_dir+'subj_SRM_fc_full.npy',subj_SRM_fc)

## load in barmaid data

In [47]:
# make ROI x TR x subject matrix
num_TR = 840
num_subs = 24
num_glasser_rois = 358
num_hc_rois = 4
num_rois = 362
barmaid_movie_data_all =np.empty([num_rois, num_TR, num_subs])

In [73]:
## extract glasser ROI data
def roi_extract(sub):
    barmaid_movie_data = np.empty([num_glasser_rois,num_TR])
    fname = analysis_dir+'confound_corrected_data/fc_data/'+subjlist_full['subj_id'][sub]+'_'+subjlist_full[movie_list[k]+'_ses'][sub]+'_task-movieviewing_'+ subjlist_full[movie_list[k]+'_place'][sub]+'_space-T1w_desc-preproc_bold_denoised_GSR_filter.nii.gz'
    epi_img_movie = nib.load(fname)
    for roi in range(num_glasser_rois):
        roi_mask = nib.load(roi_dir+subjlist_full['subj_id'][sub]+'/rois/'+roi_list['region'][roi])
        mask_movie = resample_to_img(roi_mask,epi_img_movie,interpolation='nearest')
        epi_mask_img = apply_mask(epi_img_movie, mask_movie)
        a,b = epi_mask_img.shape
        barmaid_movie_data[roi,:a] = np.mean(epi_mask_img.T,axis=0)
    return barmaid_movie_data
    

In [74]:
## run the parallelized function
start_time = time.time()
k = 0
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

barmaid_movie_data_glasser=[]
# Step 2: `pool.apply` the `perm_reinstatement()`
barmaid_movie_data_glasser= pool.map(roi_extract, range(num_subs))

# Step 3: Don't forget to close
pool.close() 

print("--- %s seconds ---" % (time.time() - start_time))

--- 5305.286900997162 seconds ---


In [75]:
## extract hippocampal ROI data
def roi_extract(sub):
    barmaid_movie_data = np.empty([num_hc_rois,num_TR])
    fname = analysis_dir+'confound_corrected_data/fc_data/'+subjlist_full['subj_id'][sub]+'_'+subjlist_full[movie_list[k]+'_ses'][sub]+'_task-movieviewing_'+ subjlist_full[movie_list[k]+'_place'][sub]+'_space-T1w_desc-preproc_bold_denoised_GSR_filter.nii.gz'
    epi_img_movie = nib.load(fname)
    for roi in range(num_hc_rois):
        roi_mask = nib.load('/Volumes/dml/ajbarnett/cartoon_recall_fmriprep_2020/hc_masks/'+subjlist_full['subj_id'][sub]+'/'+subjlist_full['subj_id'][sub]+hc_list[roi])
        mask_movie = resample_to_img(roi_mask,epi_img_movie,interpolation='nearest')
        epi_mask_img = apply_mask(epi_img_movie, mask_movie)
        a,b = epi_mask_img.shape
        barmaid_movie_data[roi,:a] = np.mean(epi_mask_img.T,axis=0)
    return barmaid_movie_data
    

In [76]:
## run the parallelized function
start_time = time.time()
k = 0
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

barmaid_movie_data_hc=[]
# Step 2: `pool.apply` the `perm_reinstatement()`
barmaid_movie_data_hc= pool.map(roi_extract, range(num_subs))

# Step 3: Don't forget to close
pool.close() 

print("--- %s seconds ---" % (time.time() - start_time))

--- 67.26163697242737 seconds ---


In [77]:
##combine HC and glasser ROIs

In [78]:
barmaid_movie_fc_data = np.empty([num_subs,num_rois,num_TR])
for i in range(num_subs): 
    barmaid_movie_fc_data[i][0:358][:] = barmaid_movie_data_glasser[i]

for i in range(num_subs): 
    barmaid_movie_fc_data[i][358:362][:] = barmaid_movie_data_hc[i]

In [135]:
barmaid_movie_fc_data = barmaid_movie_fc_data[:,:,0:832]

In [137]:
##calculate fc
subj_barmaid_movie_fc =[]
for i in range(num_subs):
    temp = barmaid_movie_fc_data[i]
    subj_barmaid_movie_fc.append(np.corrcoef(temp))

In [223]:
np.save(analysis_dir+'barmaid_movie_data_extracted_full.npy',barmaid_movie_fc_data)

In [224]:
np.save(analysis_dir+'subj_barmaid_movie_fc_full.npy',subj_barmaid_movie_fc)

In [225]:
np.save(analysis_dir+'barmaid_movie_data_extracted.npy',barmaid_movie_fc_data)

In [226]:
np.save(analysis_dir+'subj_barmaid_movie_fc.npy',subj_barmaid_movie_fc)

In [871]:
barmaid_movie_fc_data = np.load(analysis_dir+'barmaid_movie_data_extracted.npy')

## load in bluestreets data


In [149]:
# make ROI x TR x subject matrix
num_TR = 840
num_subs = 24
num_glasser_rois = 358
num_hc_rois = 4
num_rois = 362
bluestreets_movie_data_all =np.empty([num_rois, num_TR, num_subs])

In [152]:
## extract glasser ROI data
def roi_extract(sub):
    bluestreets_movie_data = np.empty([num_glasser_rois,num_TR])
    fname = analysis_dir+'confound_corrected_data/fc_data/'+subjlist_full['subj_id'][sub]+'_'+subjlist_full[movie_list[k]+'_ses'][sub]+'_task-movieviewing_'+ subjlist_full[movie_list[k]+'_place'][sub]+'_space-T1w_desc-preproc_bold_denoised_GSR_filter.nii.gz'
    epi_img_movie = nib.load(fname)
    for roi in range(num_glasser_rois):
        roi_mask = nib.load(roi_dir+subjlist_full['subj_id'][sub]+'/rois/'+roi_list['region'][roi])
        mask_movie = resample_to_img(roi_mask,epi_img_movie,interpolation='nearest')
        epi_mask_img = apply_mask(epi_img_movie, mask_movie)
        a,b = epi_mask_img.shape
        bluestreets_movie_data[roi,:a] = np.mean(epi_mask_img.T,axis=0)
    return bluestreets_movie_data
    

In [153]:
## run the parallelized function
start_time = time.time()
k = 1
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

bluestreets_movie_data_glasser=[]
# Step 2: `pool.apply` the `perm_reinstatement()`
bluestreets_movie_data_glasser= pool.map(roi_extract, range(num_subs))

# Step 3: Don't forget to close
pool.close() 

print("--- %s seconds ---" % (time.time() - start_time))

--- 5257.39177274704 seconds ---


In [154]:
## extract hippocampal ROI data
def roi_extract(sub):
    bluestreets_movie_data = np.empty([num_hc_rois,num_TR])
    fname = analysis_dir+'confound_corrected_data/fc_data/'+subjlist_full['subj_id'][sub]+'_'+subjlist_full[movie_list[k]+'_ses'][sub]+'_task-movieviewing_'+ subjlist_full[movie_list[k]+'_place'][sub]+'_space-T1w_desc-preproc_bold_denoised_GSR_filter.nii.gz'
    epi_img_movie = nib.load(fname)
    for roi in range(num_hc_rois):
        roi_mask = nib.load('/Volumes/dml/ajbarnett/cartoon_recall_fmriprep_2020/hc_masks/'+subjlist_full['subj_id'][sub]+'/'+subjlist_full['subj_id'][sub]+hc_list[roi])
        mask_movie = resample_to_img(roi_mask,epi_img_movie,interpolation='nearest')
        epi_mask_img = apply_mask(epi_img_movie, mask_movie)
        a,b = epi_mask_img.shape
        bluestreets_movie_data[roi,:a] = np.mean(epi_mask_img.T,axis=0)
    return bluestreets_movie_data
    

In [156]:
## run the parallelized function
start_time = time.time()
k = 1
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

bluestreets_movie_data_hc=[]
# Step 2: `pool.apply` the `perm_reinstatement()`
bluestreets_movie_data_hc= pool.map(roi_extract, range(num_subs))

# Step 3: Don't forget to close
pool.close() 

print("--- %s seconds ---" % (time.time() - start_time))

--- 89.22564315795898 seconds ---


In [157]:
##combine HC and glasser ROIs

In [158]:
bluestreets_movie_fc_data = np.empty([num_subs,num_rois,num_TR])
for i in range(num_subs): 
    bluestreets_movie_fc_data[i][0:358][:] = bluestreets_movie_data_glasser[i]

for i in range(num_subs): 
    bluestreets_movie_fc_data[i][358:362][:] = bluestreets_movie_data_hc[i]

In [159]:
bluestreets_movie_fc_data = bluestreets_movie_fc_data[:][:,:,0:804]

In [914]:
i=1
subj_bluestreets_movie_fc =[]

temp = bluestreets_movie_fc_data[i]
test = ma.corrcoef(ma.masked_invalid(temp))
#subj_bluestreets_movie_fc.append(ma.corrcoef(ma.masked_invalid(temp)))

In [161]:
##calculate fc
subj_bluestreets_movie_fc =[]
for i in range(num_subs):
    temp = bluestreets_movie_fc_data[i]
    
    subj_bluestreets_movie_fc.append(np.corrcoef(temp))

In [213]:
np.save(analysis_dir+'bluestreets_movie_data_extracted_full.npy',bluestreets_movie_fc_data)

In [214]:
np.save(analysis_dir+'subj_bluestreets_movie_fc_full.npy',subj_bluestreets_movie_fc)

In [17]:
bluestreets_movie_fc_data = np.load(analysis_dir+'bluestreets_movie_data_extracted_full.npy')